## Data

In [1]:
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

corpus = [
    "ăn quả nhớ kẻ trồng cây",
    "có chí thì nên"    
]
data_size = len(corpus)

# Define the max vocabulary size and sequence length
vocab_size = 15
sequence_length = 7

In [2]:
# Define tokenizer function
tokenizer = get_tokenizer('basic_english')

# Create a function to yield list of tokens
def yield_tokens(examples):
    for text in examples:
        yield tokenizer(text)

# Create vocabulary
vocab = build_vocab_from_iterator(yield_tokens(corpus),
                                  max_tokens=vocab_size,
                                  specials=["<unk>", "<pad>", "<sos_topic1>", "<sos_topic2>", "<eos>"]
                                 )
vocab.set_default_index(vocab["<unk>"])
vocab.get_stoi()

{'<unk>': 0,
 'kẻ': 8,
 '<eos>': 4,
 'cây': 6,
 'thì': 12,
 'chí': 5,
 '<pad>': 1,
 '<sos_topic1>': 2,
 '<sos_topic2>': 3,
 'có': 7,
 'nhớ': 9,
 'nên': 10,
 'quả': 11,
 'trồng': 13,
 'ăn': 14}

In [3]:
data_x = []
data_y = []

corpus[0] = '<sos_topic1> ' + corpus[0] + ' <eos>'
corpus[1] = '<sos_topic2> ' + corpus[1] + ' <eos>'

for vector in corpus:
    vector = vector.split()
    data_x.append(vector[:-1])
    data_y.append(vector[1:])  

print(data_x)
print(data_y)

[['<sos_topic1>', 'ăn', 'quả', 'nhớ', 'kẻ', 'trồng', 'cây'], ['<sos_topic2>', 'có', 'chí', 'thì', 'nên']]
[['ăn', 'quả', 'nhớ', 'kẻ', 'trồng', 'cây', '<eos>'], ['có', 'chí', 'thì', 'nên', '<eos>']]


In [4]:
# Tokenize and numericalize your samples
def vectorize(x, y, vocab, sequence_length):    
    x_ids = [vocab[token] for token in x][:sequence_length]
    x_ids = x_ids + [vocab["<pad>"]] * (sequence_length - len(x))

    y_ids = [vocab[token] for token in y][:sequence_length]
    y_ids = y_ids + [vocab["<pad>"]] * (sequence_length - len(y))
    
    return x_ids, y_ids

# Vectorize the samples
data_x_ids = []
data_y_ids = []
for x, y in zip(data_x, data_y):
    x_ids, y_ids = vectorize(x, y, vocab, sequence_length)
    data_x_ids.append(x_ids)
    data_y_ids.append(y_ids)

In [5]:
for x, y in zip(data_x_ids, data_y_ids):
    print(x)
    print(y)
    print()

[2, 14, 11, 9, 8, 13, 6]
[14, 11, 9, 8, 13, 6, 4]

[3, 7, 5, 12, 10, 1, 1]
[7, 5, 12, 10, 4, 1, 1]



In [6]:
data_x_ids = torch.tensor(data_x_ids, dtype=torch.long)
print(data_x_ids.shape)

torch.Size([2, 7])


In [7]:
data_y_ids = torch.tensor(data_y_ids, dtype=torch.long)
print(data_y_ids.shape)

torch.Size([2, 7])


## Train with full data

In [8]:
class TG_Model(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, sequence_length):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 
                                      embed_dim)
        self.recurrent = nn.RNN(embed_dim, embed_dim, batch_first=True)
        self.linear = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x,_ = self.recurrent(x)        
        x = self.linear(x)
        return x.permute(0,2,1)

model = TG_Model(vocab_size, 8, 1, sequence_length)
print(model)

TG_Model(
  (embedding): Embedding(15, 8)
  (recurrent): RNN(8, 8, batch_first=True)
  (linear): Linear(in_features=8, out_features=15, bias=True)
)


In [9]:
outputs = model(data_x_ids)
print(outputs.shape)

torch.Size([2, 15, 7])


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [11]:
# train
for _ in range(40):
    optimizer.zero_grad()
    outputs = model(data_x_ids)
    loss = criterion(outputs, data_y_ids)
    print(loss.item())
    loss.backward()
    optimizer.step()

2.819533586502075
2.4304347038269043
2.1206493377685547
1.8530852794647217
1.6029101610183716
1.3736979961395264
1.1723766326904297
0.9990950226783752
0.8480130434036255
0.719149649143219
0.6101356148719788
0.5171350836753845
0.43700775504112244
0.36806657910346985
0.30909064412117004
0.2588804066181183
0.2160690575838089
0.17948392033576965
0.14807094633579254
0.12067169696092606
0.09920600801706314
0.0841754823923111
0.0723121389746666
0.06231125071644783
0.05383419245481491
0.04673413559794426
0.04084658995270729
0.03597891330718994
0.0319448821246624
0.02858615480363369
0.02577504701912403
0.023409709334373474
0.021409226581454277
0.019707996398210526
0.018252504989504814
0.016999052837491035
0.015911592170596123
0.014961193315684795
0.01412444468587637
0.013382856734097004


In [12]:
outputs = model(data_x_ids)
#print(outputs)
print(torch.argmax(outputs, axis=1))

tensor([[14, 11,  9,  8, 13,  6,  4],
        [ 7,  5, 12, 10,  4,  1,  1]])


In [13]:
data_y_ids

tensor([[14, 11,  9,  8, 13,  6,  4],
        [ 7,  5, 12, 10,  4,  1,  1]])

## Inference

In [14]:
promt = '<sos_topic2> có'
promt = promt.split()
promt_ids = [vocab[token] for token in promt][:sequence_length]
promt_ids = promt_ids + [vocab["<pad>"]] * (sequence_length - len(promt))

print(promt_ids)

[3, 7, 1, 1, 1, 1, 1]


In [15]:
for i in range(sequence_length - len(promt)):
    promt_tensor = torch.tensor(promt_ids, dtype=torch.long).reshape(1, -1)
    outputs = model(promt_tensor)
    outputs = torch.argmax(outputs, axis=1)   
    next_id = outputs[0][len(promt)+i-1]

    promt_ids[len(promt)+i] = next_id.item()
    print(promt_ids)

[3, 7, 5, 1, 1, 1, 1]
[3, 7, 5, 12, 1, 1, 1]
[3, 7, 5, 12, 10, 1, 1]
[3, 7, 5, 12, 10, 4, 1]
[3, 7, 5, 12, 10, 4, 13]
